In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('image', cmap='gray')
mpl.rcParams["image.interpolation"] = 'none'
%matplotlib inline
import matplotlib.animation

from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import random
import time

%config InlineBackend.figure_format = 'retina'
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D
from csbdeep.utils import Path, normalize
import skimage
from tifffile import imsave, imread
from glob import glob

np.random.seed(42)
lbl_cmap = random_label_cmap()

# BUILD TRAIN DATASET

In [ ]:
def get_frame(cap, frame, x1, y1, x2, y2, w, h, preprocess):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    ret, image = cap.read()
    if preprocess:
        npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        alpha = Image.new('L', (w, h), 0)
        draw = ImageDraw.Draw(alpha)
        draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
        npAlpha = np.array(alpha)
        npImage = npImage*npAlpha
        ind = np.where(npImage == 0)
        npImage[ind] = npImage[200, 200]
        npImage = npImage[y1:y2, x1:x2]
        return npImage
    elif not preprocess:
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        raise ValueError("preprocess must be a boolean")

In [ ]:
trajectories = pd.read_parquet('./49b_1r/old/tracking_data/hough/tracking_hough_trackpy_linking.parquet')
# rename column to be consistent with the other datasets
trajectories.rename(columns={'d':'r'}, inplace=True)
# remove rows with NaN values in the radius column
trajectories.dropna(subset=['r'], inplace=True)
display(trajectories.head())

In [ ]:
video = cv2.VideoCapture('/Users/matteoscandola/MasterThesis/tracking/data/49b1r.mp4')
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, first_frame = video.read()

In [ ]:
# sample x frames from the hough-method tracking dataset
n_samples = 1000
np.random.seed(42)
sample_frames = np.sort(np.random.choice(trajectories.frame.unique().astype(int), size=n_samples, replace=False))

for frame in tqdm(sample_frames):
    frame_img = get_frame(video, frame, 55, 55, 880, 880, 920, 960, True)
    imsave(f"/Users/matteoscandola/MasterThesis/tracking/train_49b-1r/image/49b1r_frame{frame}.tif", frame_img)
    df = trajectories.loc[(trajectories.frame == frame), ["x", "y", "r"]]
    image = np.zeros((880 - 55, 880 - 55), dtype=np.uint8)
    count = 0
    for _, droplet in df.iterrows():
        x, y, radius = droplet['x'], droplet['y'], droplet['r']
        cv2.circle(image, (int(x)- 55, int(y) - 55), int(radius), int(count), thickness=-1)
        count += 1
    imsave(f"/Users/matteoscandola/MasterThesis/tracking/train_49b-1r/mask/49b1r_frame{frame}.tif", image)    

# BUIL TRAIN_DATASET V2

In [ ]:
def grayscale_crop_and_resize(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
	npImage = npImage[y1:y2, x1:x2]
	return cv2.resize(npImage, (500, 500))

In [ ]:
save_path = './training_dataset/'
nDrops = 50

source_path_25b25r = "./data/25b25r-1.mp4" 
xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r = 95, 30, 535, 470
video_25b25r = cv2.VideoCapture(source_path_25b25r)
video_25b25r.set(cv2.CAP_PROP_POS_FRAMES, 0)
w_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_WIDTH))
h_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FPS))
n_frames_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"25b25r video has {n_frames_25b25r} frames with a resolution of {w_25b25r}x{h_25b25r} and a framerate of {fps_25b25r} fps")


source_path_49b1r  = "./data/49b1r.mp4"
xmin_49b1r,  ymin_49b1r,  xmax_49b1r,  ymax_49b1r  = 20, 50, 900, 930
nDrops_post_merge_49b1r = 49
video_49b1r = cv2.VideoCapture(source_path_49b1r)
video_49b1r.set(cv2.CAP_PROP_POS_FRAMES, 0)
w_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_WIDTH))
h_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FPS))
n_frames_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"49b1r video has {n_frames_49b1r} frames with a resolution of {w_49b1r}x{h_49b1r} and a framerate of {fps_49b1r} fps")

In [ ]:
print(f"25b25r video can be cropped to : {ymax_25b25r - ymin_25b25r, xmax_25b25r - xmin_25b25r}")
print(f"49b1r  video can be cropped to : {ymax_49b1r -  ymin_49b1r,  xmax_49b1r -  xmin_49b1r }")

start_49b1r = grayscale_crop_and_resize(video_49b1r, 0, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)
end_49b1r = grayscale_crop_and_resize(video_49b1r, n_frames_49b1r-1, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)
start_25b25r = grayscale_crop_and_resize(video_25b25r, 0, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)
end_25b25r = grayscale_crop_and_resize(video_25b25r, n_frames_25b25r-1, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)

fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0,0].imshow(start_49b1r)
ax[0,0].set_title('Start 49b1r')
ax[0,1].imshow(end_49b1r)
ax[0,1].set_title('End 49b1r')
ax[1,0].imshow(start_25b25r)
ax[1,0].set_title('Start 25b25r')
ax[1,1].imshow(end_25b25r)
ax[1,1].set_title('End 25b25r')
plt.tight_layout()
plt.show()

In [ ]:
np.random.seed(0)

n_samples_49b1r  = 100
n_samples_25b25r = 100
n_samples = n_samples_49b1r + n_samples_25b25r

# 30000
sample_frames_49b1r  = np.sort(np.array(random.sample(range(30000 ), n_samples_49b1r )), axis=0)
sample_frames_25b25r = np.sort(np.array(random.sample(range(n_frames_25b25r), n_samples_25b25r)), axis=0)

video_sample_49b1r = np.zeros((n_samples_49b1r, 500, 500))
for i in tqdm(range(n_samples_49b1r)):
    video_sample_49b1r[i] = grayscale_crop_and_resize(video_49b1r, sample_frames_49b1r[i], xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)


video_sample_25b25r = np.zeros((n_samples_25b25r, 500, 500))
for i in tqdm(range(n_samples_25b25r)):
    video_sample_25b25r[i] = grayscale_crop_and_resize(video_25b25r, sample_frames_25b25r[i], xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)

In [ ]:
def get_frame_hough(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
	alpha = Image.new('L', (w, h), 0)
	draw = ImageDraw.Draw(alpha)
	draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
	npAlpha = np.array(alpha)
	npImage = npImage*npAlpha 
	ind = np.where(npImage == 0)
	npImage[ind] = npImage[200, 200]
	npImage = npImage[y1:y2, x1:x2]
	return npImage #normalize(npImage)
	
def hough_loc_frame(correct_n, frame, img, parameters):
	found_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **parameters)
	if found_circles is not None:
		return np.hstack((found_circles[0], (np.ones((found_circles.shape[1], 1), dtype=int)*frame),\
							np.ones((found_circles.shape[1], 1), dtype=int)*found_circles.shape[1]))
	else:
		return np.hstack((np.ones((1, 3))*-1, np.array([[frame, 0]])))
		
def hough_feature_location(sample_frames, correct_n, params):
	temp = []
	for frame in tqdm(sample_frames):
		img = get_frame_hough(video, frame, xmin, ymin, xmax, ymax, w, h)
		temp.append(hough_loc_frame(correct_n, frame, img, params))
	
	temp_df = pd.DataFrame(np.concatenate([arr for arr in temp]), columns = ["x", "y", "d", "frame", "nDroplets"])
	temp_df["frame"] = temp_df["frame"].astype(int)
	temp_df["nDroplets"] = temp_df["nDroplets"].astype(int)
	err_frames = temp_df.loc[temp_df.nDroplets != correct_n].frame.unique().astype(int)
	loss = err_frames.shape[0]/sample_frames.shape[0]
	return temp_df, err_frames, loss

def optimize_params(x, *args):
	frames, correct_n = args
	params = {"dp":x[0], "minDist":int(x[1]), "param1":x[2], "param2":x[3], "minRadius":int(x[4]), "maxRadius":int(x[5])}
	errs = 0
	for i in tqdm(frames):
		img = get_frame_hough(video, i, xmin, ymin, xmax, ymax, w, h)
		found_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **params)
		if (found_circles is not None) and (found_circles.shape[1] == correct_n):
			pass
		else:
			errs += 1
			
	loss = errs/frames.shape[0]
	a = [loss, x[0], int(x[1]), x[2], x[3], int(x[4]), int(x[5])]
	print(a)
	return loss

In [ ]:
test_params = {"dp":1.5, "minDist":3, "param1":50, "param2":0.8, "minRadius":3, "maxRadius":20}

#temp_df, err_frames, loss = hough_feature_location(.......

# TRAIN

In [ ]:
X = sorted(glob("./train_49b-1r/image/*.tif"))
Y = sorted(glob("./train_49b-1r/mask/*.tif"))
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y))

X = list(map(imread,X))
Y = list(map(imread,Y))
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]

axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
Y = [fill_label_holes(y) for y in tqdm(Y)]

In [ ]:
assert len(X) > 1, "not enough training data"
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train] 
print('number of images: %3d' % len(X))
print('- training:       %3d' % len(X_trn))
print('- validation:     %3d' % len(X_val))

In [ ]:
def plot_img_label(img, lbl, img_title="image", lbl_title="label", **kwargs):
    fig, (ai,al) = plt.subplots(1,2, figsize=(12,5), gridspec_kw=dict(width_ratios=(1.25,1)))
    im = ai.imshow(img, cmap='gray', clim=(0,1))
    ai.set_title(img_title)    
    fig.colorbar(im, ax=ai)
    al.imshow(lbl, cmap=lbl_cmap)
    al.set_title(lbl_title)
    plt.tight_layout()

In [ ]:
i = min(9, len(X)-1)
img, lbl = X[i], Y[i]
assert img.ndim in (2,3)
img = img if (img.ndim==2 or img.shape[-1]==3) else img[...,0]
plot_img_label(img,lbl)
None;

In [ ]:
gputools_available()

In [ ]:
# 32 is a good default choice (see 1_data.ipynb)
n_rays = 32

# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = False and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (2,2)

conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = use_gpu,
    n_channel_in = n_channel,
)
print(conf)
vars(conf)

if use_gpu:
    from csbdeep.utils.tf import limit_gpu_memory
    # adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
    limit_gpu_memory(0.8)
    # alternatively, try this:
    # limit_gpu_memory(None, allow_growth=True)

In [ ]:
timestamp = time.time()
model = StarDist2D(conf, name=f'stardist_{timestamp}', basedir='models')

In [ ]:
median_size = calculate_extents(list(Y), np.median)
fov = np.array(model._axes_tile_overlap('YX'))
print(f"median object size:      {median_size}")
print(f"network field of view :  {fov}")
if any(median_size > fov):
    print("WARNING: median object size larger than field of view of the neural network.")

In [ ]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

In [ ]:
# plot some augmented examples
img, lbl = X[0],Y[0]
plot_img_label(img, lbl)
for _ in range(3):
    img_aug, lbl_aug = augmenter(img,lbl)
    plot_img_label(img_aug, lbl_aug, img_title="image augmented", lbl_title="label augmented")

In [ ]:
%load_ext tensorboard

In [ ]:
quick_demo = False

if quick_demo:
    print (
        "NOTE: This is only for a quick demonstration!\n"
        "      Please set the variable 'quick_demo = False' for proper (long) training.",
        file=sys.stderr, flush=True
    )
    model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter,
                epochs=2, steps_per_epoch=10)

    print("====> Stopping training and loading previously trained demo model from disk.", file=sys.stderr, flush=True)
    model = StarDist2D.from_pretrained('2D_demo')
else:
    model.train(X_trn, Y_trn, validation_data=(X_val,Y_val), augmenter=augmenter, epochs=150, steps_per_epoch=100)
None;